# ${\mathbb{NAMES:}}$
1- ${Yuval\ Kaver}$, id: 329

2- ${Matan\ Ginzburg}$, id: 215

In [1]:
import numpy as np
import scipy
from numpy.linalg import pinv
np.set_printoptions(suppress=True,precision=5)

# Fundamental Matrix and Homographies

Prove that for any homography $H$ from image 1 to image 2, through a plane that does not contain the camera centers, we have:
$F \sim [e']_x H$. 

Proof: <br> 
define $(p,p')$ as a matching pair of points in $I_1$ and $I_2$ <br>
for every homography $H$ from $I_1$ to $I_2$ that satisfies the conditions mentioned above,<br>
$$
p' \sim Hp
$$ <br>
we also know that $F$ transforms $p$ from $I_1$ to the corresponding epipolar line in $I_2$,<br>
$$
Fp \sim [e']_x p'\\
\downarrow \\
\forall p : Fp \sim [e']_x p' \sim [e']_x H p \\
\forall p : Fp \sim [e']_x H p \\
$$
since this holds for $\forall p \in I_1$ we get :
$$
F \sim [e']_xH
$$

# Essential Matrix

In [2]:
#creating 4 cameras
K1 = np.array([[1, 0, 0],
              [0, 1, 0],
              [0, 0, 1]])
K2 = np.array([[10, 0, 23],
              [0, 95, 4],
              [0, 0, 1]])
K3 = np.array([[4, 0, 1],
              [0, 2, 5],
              [0, 0, 6]])
K4 = np.array([[5.2, 0, 0.4],
              [0, 4.5, 15],
              [0, 0, 10]])
R2 = np.array([[1,0,0],
              [0,0.5,-0.8660254],   
              [0,0.8660254,0.5]])
R3 = np.array([[0.93969262, 0.17101007, 0.29619813],
             [0.0, 0.8660254, -0.5],
             [-0.34202014, 0.46984631, 0.81379768]])
R4 = np.array([[0.70710678, -0.70710678, 0.0],
             [0.35355339, 0.35355339, -0.8660254],
             [0.61237244, 0.61237244, 0.5]])

t2 = np.array([1, 5, 2]).reshape(3, 1)
t3 = np.array([4, 2, 3]).reshape(3, 1)
t4 = np.array([0.3, 8, 1.7]).reshape(3, 1)

a = np.zeros((3,1))

M1 = np.hstack(( K1 , a ))
M2 = np.hstack(( K2@R2 , K2@t2 ))
M3 = np.hstack(( K3@R3 , K3@t3 ))
M4 = np.hstack(( K4@R4 , K4@t4 ))

In [3]:
def essential_matrix_from_motion(R,t):
    tt = t.reshape(3)
    _t_ = np.array([
        [0,-tt[2],tt[1]],
        [tt[2],0,-tt[0]],
        [-tt[1],tt[0],0]])
    E = _t_ @ R
    return E

In [4]:
E = essential_matrix_from_motion(R2,t2)
print (E)

[[ 0.       3.33013  4.23205]
 [ 2.      -0.86603 -0.5    ]
 [-5.       0.5     -0.86603]]


In [5]:
def motion_from_essential_matrix(E):
    U , sigma , V_T= np.linalg.svd(E)
    sigma = np.diag(sigma)
    sigma[2,2] = 0
    
    W = np.array([[0,-1,0],     #מטריצת סיבוב
              [1,0,0],
              [0,0,1]])

    Z = np.diag([1,1,0])@W         #מכפלה וקטורית בבסיס הסטנדרטי יחסית לוקטור טרנספוז (0,0,1)

    tt = U@Z@U.T
    R1 = U@W@V_T
    R2 = U@W.T@V_T
    t = np.array([tt[1][2], tt[2][0],tt[0][1]]).T

    if np.linalg.det(R1) == 1 :
        return R1,t
    return R2,t

In [6]:
def check_essential_matrix(A):
    '''
    Given a 3x3 matrix, the function checks if it is an essential matrix. 
    '''
    _ , sigma , _= np.linalg.svd(A)
    epsilon = 0.1

    if ( abs(sigma[0]-sigma[1]) <epsilon  and sigma[2] < epsilon ):      #should sigma[0] != 0      as well???/
        return True
    return False


In [7]:
#check of essential matrix functions
E = essential_matrix_from_motion(R3,t3)

R3_recreated,t3_recreated = motion_from_essential_matrix(E)

E_recreated = essential_matrix_from_motion(R3_recreated,t3_recreated)

print("real E\n",E)
normal = E[2,2]/E_recreated[2,2]
print("recreated E:\n",normal*E_recreated) 
print("\nreal R and t:")
print("R = ", R3)
print("t = ",t3.T)
print("recreated R and t (after normalizing):")
print("R = ",R3_recreated)              
print("t = ",t3_recreated*t3[2]/t3_recreated[2])

print("\ncheck E.M for real E:",check_essential_matrix(E))
print("check E.M for recreated E:",check_essential_matrix(E_recreated))
defenetily_not_essential = np.array([[1, 15, 24],
              [14, 53, 1],
              [765, 87, 57]])
print("check E.M for random E:",check_essential_matrix(defenetily_not_essential))

real E
 [[-0.68404 -1.65838  3.1276 ]
 [ 4.18716 -1.36636 -2.3666 ]
 [-1.87939  3.12208 -2.5924 ]]
recreated E:
 [[-0.68404 -1.65838  3.1276 ]
 [ 4.18716 -1.36636 -2.3666 ]
 [-1.87939  3.12208 -2.5924 ]]

real R and t:
R =  [[ 0.93969  0.17101  0.2962 ]
 [ 0.       0.86603 -0.5    ]
 [-0.34202  0.46985  0.8138 ]]
t =  [[4 2 3]]
recreated R and t (after normalizing):
R =  [[ 0.93969  0.17101  0.2962 ]
 [-0.       0.86603 -0.5    ]
 [-0.34202  0.46985  0.8138 ]]
t =  [4. 2. 3.]

check E.M for real E: True
check E.M for recreated E: True
check E.M for random E: False


# Trifocal tensor

In [8]:
#make points with M1,M2,M3
n = 10
points = abs(np.random.randn(n,3)*5)
points = np.hstack((points,np.ones((n,1))))
p1 = np.zeros((n,3))
p2 = np.zeros((n,3))
p3 = np.zeros((n,3))
p4 = np.zeros((n,3))
for i in range(n):
    p1[i] = M1@points[i].T
    p1[i] = p1[i]/p1[i][2]
    p2[i] = M2@points[i].T
    p2[i] = p2[i]/p2[i][2]
    p3[i] = M3@points[i].T
    p3[i] = p3[i]/p3[i][2]
    p4[i] = M4@points[i].T
    p4[i] = p4[i]/p4[i][2]
s = np.array([[-1,0,p2[0][0]],
              [0,-1,p2[0][1]]])
r = np.array([[-1,0,p3[0][0]],
              [0,-1,p3[0][1]]])
p = np.zeros((n,3,3))
p[:,:,0] = p1
p[:,:,1] = p2
p[:,:,2] = p3
p_4cam = np.zeros((n,3,4))
p_4cam[:,:,0] = p1
p_4cam[:,:,1] = p2
p_4cam[:,:,2] = p3
p_4cam[:,:,3] = p4

In [9]:
def normalize_points(points):
    return (points.T/points[:, -1].T).T

In [10]:
def cameras_to_tensor(M1,M2,M3):
    '''
    Given three cameras under the form [I,0], [A,a_4], [B,b_4], the function computes the trifocal tensor of this triplet of cameras. 
    '''
    T = np.zeros((3,3,3))
    for i in range(3):
        for j in range(3):
            for k in range(3):
                T[i][j][k] = M2[j][3]*M3[k][i] - M2[j][i]*M3[k][3]
    return T

Incidence measure: $T_i^{jk} s_j r_k p^i$. 

In [11]:
def incidence_measure(T,p,s,r):
    '''
    Given the trifocal tensor T and a point p in the first image, a line s in the second image and a line r in the third image,
    the functions computes the indicence measure.
    '''
    incidence = 0
    for i in range(3):
        for j in range(3):
            for k in range(3):
                incidence += T[i][j][k]*s[j]*r[k]*p[i]
    return abs(incidence)

In [12]:
#cameras to tensor test
T_cameras = cameras_to_tensor(M1,M2,M3)
incidence = 0
for qq in range(n):
    s = np.array([[-1,0,p2[qq][0]],
              [0,-1,p2[qq][1]]])
    r = np.array([[-1,0,p3[qq][0]],
              [0,-1,p3[qq][1]]])
    for i in range(2):
        incidence += incidence_measure(T_cameras,p1[qq],s[i],r[i])
print(incidence)


8.485656621814996e-12


In [13]:
def fundamental_matrix_from_tensor(T,l1,l2):
    H1 = np.zeros((3,3))
    H2 = np.zeros((3,3))
    for i in range(3):
        for j in range(3):
            for k in range(3):
                H1[j][i] += T[i][j][k]*l1[k]
                H2[j][i] += T[i][j][k]*l2[k]
    A = np.zeros((12,9))
    count = 0
    for h in [H1,H2]:
        for j in range(3):
            for i in range(j,3):
                A[count][i] += h[0][j]
                A[count][j] += h[0][i]
                A[count][i+3] += h[1][j]
                A[count][j+3] += h[1][i]
                A[count][i+6] += h[2][j]
                A[count][j+6] += h[2][i]
                count+=1
    U , _ , __= np.linalg.svd(A.T@A)
    f = np.reshape(U[:,-1],(3,3))
    U , sigma , V= np.linalg.svd(f)
    sigma = np.diag(sigma)
    sigma[2,2] = 0          #we force rank of F to be 2
    F = U@sigma@V  
    return F

In [14]:
#functions from previous assignments
def projective_reconstruction(M2,pixel1,pixel2):
    A = np.zeros((4,4))
    A[0] = [-1,0,pixel1[0],0]
    A[1] = [0,-1,pixel1[1],0]
    A[2] = pixel2[1]*M2[2] - M2[1]
    A[3] = pixel2[0]*M2[2] - M2[0]
    ATME = A.T@A
    U , _ , __= np.linalg.svd(ATME)
    P = U[:,-1]
    return P/P[3]
def linear_calibration(p,P):
    '''
    Input: 
        p: nx2 array (pixels in an image)
        P: nx3 array (points in 3d)
        Assumption: The two arrays have corresponding order. 
    Output: 
        M: camera matrix 
    '''
    A = np.zeros((2*len(P),12))
    for i in range(len(P)):
        A[2*i] = [-P[i][0],-P[i][1],-P[i][2],-1,0,0,0,0,P[i][0]*p[i][0],P[i][1]*p[i][0],P[i][2]*p[i][0],p[i][0]]
        A[2*i+1] = [0,0,0,0,-P[i][0],-P[i][1],-P[i][2],-1,P[i][0]*p[i][1],P[i][1]*p[i][1],P[i][2]*p[i][1],p[i][1]]
    U , _ , __= np.linalg.svd(A.T@A)
    M1 = np.reshape(U.T[-1],(3,4))
    return M1
def vector_mult_matrix(v):
    return np.array([[0,-v[2],v[1]],
                     [v[2],0,-v[0]],
                     [-v[1],v[0],0]])

In [15]:
def tensor_cameras(T,p1,p2,l1,l2):
    '''
    Given the trifocal tensor T and matching points in the two images p1,p2 and two lines in the 3rd image, 
    the function computes the three projective camera matrices in the same coordinate system.
    '''
    F = fundamental_matrix_from_tensor(T,l1,l2)
    epipolar_tag = np.array(scipy.linalg.null_space(F.T))[:, 0]  # need to change it to correct epipolar
    M2 = vector_mult_matrix(epipolar_tag)@F
    M2 = np.hstack((M2, np.array([epipolar_tag]).T))
    P = np.zeros((n,4))
    for i in range(n):
        P[i] = projective_reconstruction(M2,p1[i],p2[i])
    p3_proj = np.zeros((n,3))
    for m in range(n):
        for k in range(3):
            for j in range(3):
                for i in range(3):
                    l = np.array([-1,0,p2[m][0]])
                    p3_proj[m][k] += T[i][j][k]*p1[m][i]*l[j]
    p3_proj = normalize_points(p3_proj)
    M3 = linear_calibration(p3_proj,P)
    return M1,M2,M3,P


In [16]:
#tensor cameras test
l1 = np.array([-1,0,p3[0][0]])
l2 = np.array([0,-1,p3[3][1]])
M1_projective,M2_projective,M3_projective,P_projective = tensor_cameras(T_cameras,p1,p2,l1,l2)
error = np.zeros(3)
p1_proj = normalize_points((M1_projective@P_projective.T).T)
p2_proj = normalize_points((M2_projective@P_projective.T).T)
p3_proj = normalize_points((M3_projective@P_projective.T).T)
error[0] += np.sum(p1 - p1_proj)
error[1] += np.sum(p2 - p2_proj)
error[2] += np.sum(p3 - p3_proj)
print("error between p1 and M1@P: ",error[0])
print("error between p2 and M2@P: ",error[1])
print("error between p3 and M3@P: ",error[2])

error between p1 and M1@P:  4.99184027447086e-14
error between p2 and M2@P:  5.931255486757436e-12
error between p3 and M3@P:  1.2245759961615477e-13


In [17]:
def epsilon(i,j,k):
    if i==j or i==k or j==k : return 0
    m_replace = 0
    count =  0

    if i>j and k<j : 
        m_replace=1 #(3,2,1) 

    elif i>j:
        m_replace += 1 
        if i>k:
            m_replace += 1

    elif j>k:
        m_replace += 1
        if i>k:
            m_replace += 1
    

    if m_replace%2:     #if m_replace%2
        return -1
    else:
        return 1


In [18]:
def tensor_linear_computation(p):
    '''
    Input: p: nx2x3 array. For i, p[i,...] is a 2x3 matrix which contains 3 matching points in the images 1,2,3. 
    Output: Trifocal tensor
    '''
    A = np.zeros((9*n,27))
    for m in range(n):  
        for r in range(3):
            for s in range(3):          #row = 9*m + 3*r + s
                for i in range(3): 
                    for j in range(3):
                        for k in range(3):      #sec = 9*i + 3*j + k
                            for a in range(3):              #for p'
                                for b in range(3):          #for p''
                                    A[9*m+r*3+s][i*9+j*3+k] += p[m][i][0] * p[m][a][1] * p[m][b][2] * epsilon(a+1,j+1,r+1) * epsilon(b+1,k+1,s+1)
                                                              
    A_T = A.T@A
    U , _ , __= np.linalg.svd(A_T)
    tensor = U[:,-1]
    T1 = np.zeros((3,3,3))
    for i in range(3): 
        for j in range(3):
            for k in range(3):
                T1[i][j][k] = tensor[i*9+j*3+k]

    tensor = tensor.reshape(3,3,3)
    return tensor          

In [19]:
#tensor linear computation test
T_linear = tensor_linear_computation(p)
incidence = 0
for qq in range(n):
    s = np.array([[-1,0,p2[qq][0]],
              [0,-1,p2[qq][1]]])
    r = np.array([[-1,0,p3[qq][0]],
              [0,-1,p3[qq][1]]])
    for i in range(2):
        incidence += incidence_measure(T_linear,p1[qq],s[i],r[i])
print(incidence)


1.6086213524418858e-11


In [20]:
def tensor_cameras_stack(T,p1,p2,P): #stack only calculates the added camera
    #this doesnt change our projective reconstruction or other cameras to avoid stacking errors
    '''
    Given the trifocal tensor T and matching points in the two images p_n-2,p_n-1 and projective reconstruction points P, 
    the function computes the projective camera matrix in the same coordinate system.
    '''
    p_proj = np.zeros((n,3))
    for m in range(n):
        for k in range(3):
            for j in range(3):
                for i in range(3):
                    l = np.array([-1,0,p2[m][0]])
                    p_proj[m][k] += T[i][j][k]*p1[m][i]*l[j]
    p_proj = normalize_points(p_proj)
    M = linear_calibration(p_proj,P)
    return M

In [21]:
def camera_threading(p):
    '''
    Input: p: nx2x4 array. For i, p[i,...] is a 2x4 matrix which contains 4 matching points in the images 1,2,3,4.  
    Output: [M1,M2,M3,M4]: four (projective) cameras in the same coordinate system.
    '''
    T1 = tensor_linear_computation(p[:,:,:3])
    l1 = np.array([-1,0,p[4,0,2]])
    l2 = np.array([0,-1,p[1,1,2]])
    M1, M2, M3, P= tensor_cameras(T1,p[:,:,0],p[:,:,1],l1,l2)
    T2 = tensor_linear_computation(p[:,:,1:4])
    M4 = tensor_cameras_stack(T2,p[:,:,1],p[:,:,2],P)
    return M1,M2,M3,M4,P
    

In [22]:
#camera threading test
M1_projective,M2_projective,M3_projective,M4_projective,P_projective = camera_threading(p_4cam)
error = np.zeros(4)
p1_proj = normalize_points((M1_projective@P_projective.T).T)
p2_proj = normalize_points((M2_projective@P_projective.T).T)
p3_proj = normalize_points((M3_projective@P_projective.T).T)
p4_proj = normalize_points((M4_projective@P_projective.T).T)
error[0] += np.sum(p1 - p1_proj)
error[1] += np.sum(p2 - p2_proj)
error[2] += np.sum(p3 - p3_proj)
error[3] += np.sum(p4 - p4_proj)
print("error between p1 and M1@P: ",error[0])
print("error between p2 and M2@P: ",error[1])
print("error between p3 and M3@P: ",error[2])
print("error between p3 and M3@P: ",error[3])

error between p1 and M1@P:  -1.0308295883554308e-11
error between p2 and M2@P:  4.057792324374532e-09
error between p3 and M3@P:  -4.2264146626891375e-09
error between p3 and M3@P:  3.959414135723538e-07


# Autocalibration

In [23]:
def check_kruppa(F,omega1,omega2):
    '''
    Given F and the projections of the absolute conic onto the images, the function checks that Kruppa equation holds. 
    '''
    e = scipy.linalg.null_space(F).reshape(3)
    e_x =  vector_mult_matrix(e)

    #Kruppa equations: fir ~ sec
    fir = F.T@omega2@F          
    sec = e_x@omega1@e_x

    fir2 = fir/fir[2][2] #normilizing each expression
    sec2 = sec/sec[2][2]

    return np.allclose(fir2,sec2)

In [24]:
M1_ = M1[:,:3]
m1 = M1[:,3]
M2_ = M2[:,:3]
m2 = M2[:,3]

e2 = (-M2_@np.linalg.inv(M1_)@m1 + m2).reshape(3)

e2_x =  vector_mult_matrix(e2)

F = e2_x@M2_@np.linalg.inv(M1_)

omeg1= K1@K1.T
omeg2= K2@K2.T

print("does kruppa work: ",check_kruppa(F,omeg1,omeg2))

does kruppa work:  True
